In [1]:
# LO HE HECHO EN COLAB

# La práctica deberá realizarse de manera individual y debe entregarse en la plataforma de Instituto BME.

# Para cada ejercicio: 1) Entregad un fichero en ipython notebook (.ipynb) ejecutable en Colab, y 
# 2) cada notebook ha de incluir un comentario sobre el ejercicio y sus resultados.

# 7) Utiliza el modelo de Q-learning funcional con distintas redes neuronales para resolver el entorno de RL-Bolsa.
# CONCLUSION: HE INTENTADO DE TODO, COMO SE VE MAS ABAJO PERO NO ME SALE.

In [2]:
# tengo en los apuntes que Valero dijo en clase sobre este punto que se podia usar
# los modelos usados en el ejercicio 5
# para resolver este entorno.

# el tema es que valores de discount_factor, alpha, y epsilon como inicialaes y 
# hacer lo mismo que en el ejercio 4 para ver unos pre-optimos

In [9]:
#1 Copio el modelo del ejercicio 6

# Libraries
# Basics
import numpy as np
import pandas as pd


# Visualization
import matplotlib.pyplot as plt

# Reinforcement Learning
import gym
from gym import spaces

# DATA
# Import data
data = pd.read_csv("googl.us.txt", delimiter=",")

# d. Incluir ruido en los datos.
# considero que meter ruido en los datos es que 
# algunos datos no se tenian y entocnes se rellenaban con los del dia anterior
data = data['Open']
# viejo
#Genero en y un 10% de datos a 0
cuantos_random = int(data.shape[0]*0.1)
a_acero_array = []
for i in range(0,cuantos_random):
    a_cero = np.random.randint(0, data.shape[0])
    a_acero_array.append(a_cero)
data[a_acero_array] = np.nan
# pd.DataFrame(data).plot(figsize=(20,10))
data.fillna(method='ffill', inplace=True)
data.fillna(method='bfill', inplace=True)
# pd.DataFrame(data).plot(figsize=(20,10))

# vector_incertidumbre = np.full(shape = y.shape[0], fill_value = 100, dtype=None)
# vector_incertidumbre[a_acero_array] = 0
# pd.DataFrame(vector_incertidumbre).plot(figsize=(20,10))
# y, y.shape, vector_incertidumbre, vector_incertidumbre.shape

# DEFINITION OF ENVIRONMENT
# original de los notebook de Valero con Porcentaje comision
class OneStockEnv(gym.Env):
    '''
    Reinforcement Leaning environment representing a Stock Market with a single stock.
    '''
    def __init__(self, 
                 data,
                 time_skip    = 1,
                 time_horizon = 10,
                 # comision = 0 #a.  Meter comisiones. Porcentqaje comision
                 porcentaje_comision = 0.0007 #a.  Meter comisiones. Porcentqaje comision
                 ):
        '''
        Initialization of the environment.
        
        Args:
            data
            time_skip    (int):      number of timesteps to skip between states
        '''
        # Get the data
        self.data = data
        self.time_horizon = time_horizon
        self.time_skip = time_skip
        #self.comision = comision #a.  Meter comisiones. Porcentqaje comision
        self.porcentaje_comision = porcentaje_comision #a.  Meter comisiones. Porcentqaje comision
        # Define actions
        self.actions = {0 : 'Hold', 1 : 'Buy', 2 : 'Sell'}
        
        # Save useful values
        self.n_timesteps = (self.data.shape[0] - self.time_horizon) 
        self.n_features  = 1
        self.n_actions   = len(self.actions)
        self.state_shape = (self.time_horizon, self.n_features)
        
        # Define spaces
        self.action_space      = spaces.Discrete(self.n_actions)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=self.state_shape, dtype=np.float32)

        # Initialize the environment
        self.reset()
    
    def reset(self):
        '''
        Initalize temporal variables.
        '''
        # Intialize the values
        self.t = self.time_horizon
        self.positions_opened = []
        self.position_value = 0
        self.done = False
       

        # Define some useful debug variables
        self.history_dates = []
        self.history_actions = []
        self.history_position_values = []
        self.history_profits = []
        self.history_rewards = []
        
        # Get first state
        # initial_state = self.data[:self.t]
        # initial_state = np.array(initial_state).reshape(1,self.time_horizon,self.n_features)

        self.next_state = self.data[(self.t - self.time_horizon):self.t]
        self.next_state = np.array(self.next_state).reshape(1,self.time_horizon,self.n_features)
        
        return self.next_state

    def step(self, action):
        # Compute position value and profits
        position_value = sum([self.data[self.t] for p in self.positions_opened])
        profits        = sum([self.data[self.t] - p for p in self.positions_opened])
        
        # Hold
        if action == 0:
            # Use the profits as reward

            #VAL: Esto no sera? reward = 0
            #reward = profits
            reward = 0
        # Buy
        elif action == 1:
            # Open a new position
            self.positions_opened.append(self.data[self.t])
            
            # reward = -self.data[self.t-1]-self.comision #a.  Meter comisiones. Porcentaje comision
            reward = -self.data[self.t-1] #a.  Meter comisiones. Porcentaje comision
            reward = -(1-self.porcentaje_comision) #a.  Meter comisiones. Porcentaje comision
        # Sell
        elif action == 2:
            # Check that the agent as open positions:
            if len(self.positions_opened) > 0:
                # Close all opened positions
                self.positions_opened = []                

                # Use the profits as reward
                # reward = profits
                # VAL: Esto será el valor de venta, no?
                # reward = self.data[self.t-1]-self.comision #a.  Meter comisiones. Porcentqaje comision
                reward = -self.data[self.t-1] #a.  Meter comisiones. Porcentaje comision
                reward = -(1-self.porcentaje_comision) #a.  Meter comisiones. Porcentaje comision

            # else: # b.# - Aceptar inversion en corto.
                # Penalize selling when having no opened positions
            #     reward = -1 # b.# - Aceptar inversion en corto.

        # Update debug variables
        #self.history_dates.append(self.data.index[self.t])
        self.history_actions.append(action)
        self.history_position_values.append(position_value)
        self.history_rewards.append(reward)
        
        # Prepare the next step
        self.t += self.time_skip
        self.next_state = self.data[(self.t - self.time_horizon):self.t]
        self.next_state = np.array(self.next_state).reshape(1,self.time_horizon,self.n_features)

        print(self.t)
        # Check if we can continue operating
        if (self.t + self.time_skip) >= len(self.data):
            self.done = True

        # Return (obs, reward, done, info)
        return (self.next_state, reward, self.done, {})

    def render(self, mode = 'human', verbose = False):
        print('PROFITS: {:.3f}'.format(np.sum(self.history_rewards)))
        print('POSITION VALUE: {:.3f}'.format(self.position_value))
        plt.plot(self.next_state[0,:,0])

# creado pro mi en ejercicio 6
class OneStockEnv_vender_porcion_porc_comis(gym.Env):
    '''
    Reinforcement Leaning environment representing a Stock Market with a single stock.
    '''
    def __init__(self, 
                 data,
                 time_skip    = 1,
                 time_horizon = 10,
                 # comision = 0 #a.  Meter comisiones. Porcentaje comision
                 porcentaje_comision = 0.0007 #a.  Meter comisiones. Porcentaje comision
                 ):
        '''
        Initialization of the environment.
        
        Args:
            data
            time_skip    (int):      number of timesteps to skip between states
        '''
        # Get the data
        self.data = data
        self.time_horizon = time_horizon
        self.time_skip = time_skip
        #self.comision = comision #a.  Meter comisiones. Porcentaje comision
        self.porcentaje_comision = porcentaje_comision #a.  Meter comisiones. Porcentaje comision
        # Define actions
        self.actions = {0 : 'Hold', 1 : 'Buy', 2 : 'Sell'}
        
        # Save useful values
        self.n_timesteps = (self.data.shape[0] - self.time_horizon) 
        self.n_features  = 1
        self.n_actions   = len(self.actions)
        self.state_shape = (self.time_horizon, self.n_features)
        
        # Define spaces
        self.action_space      = spaces.Discrete(self.n_actions)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=self.state_shape, dtype=np.float32)

        # Initialize the environment
        self.reset()
        
        
        
    
    def reset(self):
        '''
        Initalize temporal variables.
        '''
        # Intialize the values
        self.t = self.time_horizon
        self.positions_opened = []
        
        self.position_value = 0
        self.done = False
        
            

        # Define some useful debug variables
        self.history_dates = []
        self.history_actions = []
        self.history_position_values = []
        self.history_profits = []
        self.history_rewards = []
        
        # Get first state
        # initial_state = self.data[:self.t]
        # initial_state = np.array(initial_state).reshape(1,self.time_horizon,self.n_features)

        self.next_state = self.data[(self.t - self.time_horizon):self.t]
        self.next_state = np.array(self.next_state).reshape(1,self.time_horizon,self.n_features)
        
        return self.next_state

    def step(self, action,porcion): # porcion de venta
        # Compute position value and profits
        position_value = sum([self.data[self.t] for p in self.positions_opened])
        profits        = sum([self.data[self.t] - p for p in self.positions_opened])

        # Hold
        if action == 0:
            # Use the profits as reward

            #VAL: Esto no sera? reward = 0
            #reward = profits
            reward = 0
        # Buy
        elif action == 1: 
            
            

            # c. si compro afecto un 0.01% el valor del precio de compra hacia arriba( por el procentaje de lo que voy a comprar)
            print(f'precio de compra original:{self.data[self.t]}') #nuevo. c- Modificar el precio de salida teniendo en cuenta el impacto
            self.data[self.t] = self.data[self.t]*porcion*(1+0.0001) #nuevo c- Modificar el precio de salida teniendo en cuenta el impacto
            print(f'precio de compra afectado final:{self.data[self.t]}') #nuevo c- Modificar el precio de salida teniendo en cuenta el impacto       
           
            # Open a new position
            self.positions_opened.append(self.data[self.t]) # compro una proporcion que pueden ser 2 acioens o 3, etc           
            # reward = -self.data[self.t-1]-self.comision #a.  Meter comisiones. Porcentaje comision
            reward = -self.data[self.t-1] * porcion #a.  Meter comisiones. Porcentaje comision # compro una proporcion que pueden ser 2 acciones o 3, etc
            reward = reward*(1-self.porcentaje_comision) #a.  Meter comisiones. Porcentaje comision
            # en_open_o_en_close = 1
            
        # Sell
        elif action == 2:
            # Check that the agent as open positions:
            if len(self.positions_opened) > 0:
                # Close all opened positions
                a_vender = sum(self.positions_opened) # VENDER PORCION
                self.positions_opened = []
               
                # c. si vendo afecto un 0.01% el valor del precio de venta hacia abajo( por el porcentaje de lo que voy a comprar)
                # pero en este caso entiendo que es t-1
                print(f'precio de venta original:{self.data[self.t-1]}') #nuevo
                self.data[self.t-1] = self.data[self.t-1]*porcion*(1-0.0001) #nuevo
                print(f'precio de venta afectado final:{self.data[self.t-1]}') #nuevo                
               
               
               
                # Use the profits as reward
                # reward = profits
                # reward = self.data[self.t-1]-self.comision #a.  Meter comisiones. Porcentqaje comision # porcentaje de lo que vendo
                reward = -self.data[self.t-1]*porcion #a.  Meter comisiones. Porcentaje comision
                reward = reward*(1-self.porcentaje_comision) #a.  Meter comisiones. Porcentaje comision
               
                # print(-self.data[self.t-1]*(1-porcion))
               
                # print(reward*(1-porcion))
                print(f'total de venta: {a_vender}')
                porcion_no_vendida = a_vender*(1-porcion)# me quedo con la porcion que no he vendido
                print(f'vendido: {a_vender*porcion}')
                self.positions_opened.append(porcion_no_vendida)
                
          
                
                
                

            # else: # b.# - Aceptar inversion en corto.
            #    # Penalize selling when having no opened positions
            #    reward = -1 # b.# - Aceptar inversion en corto.

        # Update debug variables
        #self.history_dates.append(self.data.index[self.t])
        self.history_actions.append(action)
        self.history_position_values.append(position_value)
        self.history_rewards.append(reward)
        
        # Prepare the next step
        self.t += self.time_skip
        self.next_state = self.data[(self.t - self.time_horizon):self.t]
        self.next_state = np.array(self.next_state).reshape(1,self.time_horizon,self.n_features)

        print(self.t)
        # Check if we can continue operating
        if (self.t + self.time_skip) >= len(self.data):
            self.done = True
        
        
        # print(self.next_state) ############
        # Return (obs, reward, done, info)
        return (self.next_state, reward, self.done, {})

    def render(self, mode = 'human', verbose = False):
        print('PROFITS: {:.3f}'.format(np.sum(self.history_rewards)))
        print('POSITION VALUE: {:.3f}'.format(self.position_value))
        plt.plot(self.next_state[0,:,0])
        
# Create environment
env = OneStockEnv(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)
# env = OneStockEnv_vender_porcion_porc_comis(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)

# env.reset()
# plt.imshow(env.render()[0])
# plt.axis('off');

In [10]:
env = OneStockEnv(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)
print(env.observation_space.shape)
env = OneStockEnv_vender_porcion_porc_comis(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)
env.observation_space.low, env.observation_space.high, env.observation_space.shape, env.observation_space.dtype, 
env.observation_space.shape

        # Define spaces
        #self.action_space      = spaces.Discrete(self.n_actions)
        #self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=self.state_shape, dtype=np.float32)

(20, 1)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(20, 1)

In [11]:
# cargo para analizar
# RL3_Stablebaselines3_SpaceInvaders.ipynb
# Esto hacía falta antes
#!pip install --upgrade --force-reinstall gym

# La opcion oficial no funciona 5/4/2023
#!pip install stable_baselines3[extra]

# Alternativa 
!pip install git+https://github.com/carlosluis/stable-baselines3@fix_tests
!pip install --upgrade --force-reinstall gym
!pip install stable_baselines3
!pip3 install gym[atari,accept-rom-license]
import gym
#from gym.wrappers import Monitor
import base64
from pathlib import Path
from IPython import display as ipythondisplay
from stable_baselines3 import A2C
from stable_baselines3.common.env_util import make_vec_env
import os

from stable_baselines3.common.evaluation import evaluate_policy

# Cargamos el entorno
# ENV_NAME = 'SpaceInvaders-v5'
ENV_NAME = 'Asterix-v5'
env = gym.make(ENV_NAME)
height, width, channels = env.observation_space.shape
actions = env.action_space.n

model = A2C("MlpPolicy", env, verbose=1) #Podemos elegir entre varias politicas, como MLPpolicy, CnnPolicy, MultiInputPolicy
model.learn(total_timesteps=25000)
model.save("a2c_Asterix")

recompensa_media, desviacion = evaluate_policy(model, model.get_env(), deterministic=True, n_eval_episodes=20)
print(f"recompensa_media:{recompensa_media:.2f} +/- {desviacion:.2f}")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/carlosluis/stable-baselines3 (to revision fix_tests) to /tmp/pip-req-build-tfi25_0e
  Running command git clone --filter=blob:none --quiet https://github.com/carlosluis/stable-baselines3 /tmp/pip-req-build-tfi25_0e
  Running command git checkout -b fix_tests --track origin/fix_tests
  Switched to a new branch 'fix_tests'
  Branch 'fix_tests' set up to track remote branch 'fix_tests' from 'origin'.
  Resolved https://github.com/carlosluis/stable-baselines3 to commit 6617e6e73cb3a70f3e88cea780ea12bed95c099e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached gym-0.26.2-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 111.1 MB/s eta 0:00:00
  Attempting uninstall: gym-notices
    Found existing installation: gym-notices 0.0.8
    Uninstalling gym-notices-0.0.8:
      Successfully uninstalled gym-notices-0.0.8
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Uninstalling cloudpickle-2.2.1:
      Successfully uninstalled cloudpickle-2.2.1
  Attempting uninstall: gym
    Found existing installation: gym 0.26.2
    Uninstalling gym-0.26.2:
      Successfully uninstalled gym-0.26.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 51.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446660 sha256=50257b5f4499498811854996f3c7ae156739f1e9fb917b24af6ca827eecc2a6e
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


/usr/local/lib/python3.10/dist-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(


NameNotFound: ignored

In [ ]:
# Instalar librerías
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

if COLAB:
  !sudo apt-get install -y xvfb ffmpeg
  !pip install -q 'gym==0.10.11'
  !pip install -q 'imageio==2.4.0'
  !pip install -q PILLOW
  !pip install -q 'pyglet==1.3.2'
  !pip install -q pyvirtualdisplay
  !pip install -q --upgrade tensorflow-probability
  !pip install -q tf-agents


# Roms de Atari ya que Gym no provee las Roms
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

!pip install ale-py

In [12]:
# 2 Busco unos preoptimos valores de discount_factor, alpha, y epsilon como hice en el ej 4

# igual al de eljercicio 4

# Install pygame and virtual display (needed in Colab)
# !pip install pygame
# !pip install pyvirtualdisplay
# !apt install xvfb
# from pyvirtualdisplay import Display
# Display("xvfb").start()
import numpy as np
import matplotlib.pyplot as plt
import gym
from tqdm.notebook import tqdm, trange
# To build and display animations
# from PIL import Image
# from IPython import display


num_episodes = 2000
rList = []
learning_rate_list = [0.5,0.7,0.9]
gamma_list = [.92,.94,.96,.98]
eps_list = [.08,.095,.105,.12]

import pandas as pd
resultados = pd.DataFrame(columns=['learning_rate','gamma','eps','sum(rList)/num_episodes','episode+1'])

for learning_rate in learning_rate_list: 
    for gamma in gamma_list:
        for eps in eps_list:
            # Initialize table with all zeros
            # Q = np.zeros([env.observation_space.n, env.action_space.n])
            ###############env.observation_space.shape[0] 20
            Q = np.zeros([env.observation_space.shape[0], env.action_space.n]) # nuevo
            # print(Q)
            # Set learning parameters
            nA = env.action_space.n

            # create list to contain total rewards
            rList = []
            episode = 0
            while sum(rList)/num_episodes < 0.05:
                episode += 1
                num_episodes = episode

                # Reset environment and get first new observation
                
                # ME SALE ESTE ERROR DE LA s IndexError: arrays used as indices must be of integer (or boolean) type en ai = np.argmax(Q[s,:])
                s = env.reset()
                # entonces me he creado esto
                # s = (env.reset()*1000).astype(int)



                rAll = 0
                # The Q-Table learning algorithm
                for _ in range(100):
                    # Choose an action by using epsilon greedy 
                    ai = np.argmax(Q[s,:])
                    
                    p = np.ones((nA,))*eps/(nA-1)
                    p[ai] = 1-eps
                    a  = np.random.choice(np.arange(len(p)), p=p)
                    # Get new state and reward from environment
                    s1, reward, terminated, truncated, info = env.step(a)
                    # Update Q-Table with new knowledge
                    Q[s,a] = Q[s,a] + learning_rate * (reward + gamma * np.max(Q[s1,:]) - Q[s,a])
                    
                    rAll += reward
                    s = s1
                    if terminated or truncated:
                        break
                # break
                rList.append(rAll)

                if (episode+1) % 10 == 0:
                    print("Episode %4d, reward %.2f" % (episode+1, sum(rList)/num_episodes))

            print("Score over time:", sum(rList)/num_episodes)
            print("Last reward:", rList[-1])

            plt.plot(rList)


            #nuevo
            resultados.loc[len(resultados .index)] = [learning_rate,gamma,eps,sum(rList)/num_episodes,episode+1] 


            print(learning_rate,gamma,eps,sum(rList)/num_episodes,episode+1)
resultados

IndexError: ignored

In [15]:
env.action_space

Discrete(3)

In [16]:
# 3
# si busco discount_factor, alpha, y epsilon como se dice en el
# notebook RL_Q_learning_paso_a_paso ya que ahi no se necesita
# env.observation_space.n

!git clone https://github.com/dennybritz/reinforcement-learning.git
import numpy as np
import pprint
import sys
sys.path.append("/content/reinforcement-learning/")
from lib.envs.gridworld import GridworldEnv
from collections import defaultdict
import itertools

# Leer el entorno
pp = pprint.PrettyPrinter(indent=2)
# env = GridworldEnv()

# Definir funcion auxiliar
def make_epsilon_greedy_policy(Q, epsilon, nA):
    """
    Creates an epsilon-greedy policy based on a given Q-function and epsilon.
    Args:
    Q: A dictionary that maps from state -> action-values.
    Each value is a numpy array of length nA (see below)
    epsilon: The probability to select a random action . float between 0 and 1.
    nA: Number of actions in the environment.
    Returns:
    A function that takes the observation as an argument and returns
    the probabilities for each action in the form of a numpy array of length nA.
    """
    def policy_fn(observation):
        print(f'observation{observation}')
        A = np.ones(nA, dtype=float) * epsilon / (nA-1)
        best_action = np.argmax(Q[observation])
        A[best_action] = (1.0 - epsilon)
        return A
    return policy_fn

# Q-LEARNING
def q_learning(env, num_episodes, discount_factor=1.0, alpha=0.5, epsilon=0.1):
    """
    Q-Learning algorithm: Off-policy TD control. Finds the optimal greedy policy
    while following an epsilon-greedy policy
    Args:
    env: OpenAI environment.
    num_episodes: Number of episodes to run for.
    discount_factor: Gamma discount factor.
    alpha: TD learning rate.
    epsilon: Chance the sample a random action. Float betwen 0 and 1.
    Returns:
    A tuple (Q, episode_lengths).
    Q is the optimal action-value function, a dictionary mapping state -> action values
    stats is an EpisodeStats object with two numpy arrays for episode_lengths and episo
    """
    # The final action-value function.
    # A nested dictionary that maps state -> (action -> action-value).
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    
    # Keeps track of useful statistics
    episode_lengths=np.zeros(num_episodes)
    episode_rewards=np.zeros(num_episodes)
    # The policy we're following
    
    
    policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
    for i_episode in range(num_episodes):
        # Print out which episode we're on, useful for debugging.
        if (i_episode + 1) % 100 == 0:
            print("\rEpisode {}/{}.".format(i_episode + 1, num_episodes), end="")
            sys.stdout.flush()
        # Reset the environment and pick the first action
        state = env.reset()
        print(f'state {state}')###########################
        # One step in the environment
        # total_reward = 0.0
        for t in itertools.count():
            # Take a step
            action_probs = policy(state)
            action = np.random.choice(np.arange(len(action_probs)), p=action_probs)
            next_state, reward, done, _ = env.step(action)
            # Update statistics
            episode_rewards[i_episode] += reward
            episode_lengths[i_episode] = t
            # TD Update
            best_next_action = np.argmax(Q[next_state])
            td_target = reward + discount_factor * Q[next_state][best_next_action]
            td_delta = td_target - Q[state][action]
            Q[state][action] += alpha * td_delta
            print(f'Q{Q}')###########################
            if done:
                break
            state = next_state
    return Q, episode_lengths, episode_rewards
# Paso a paso
#Parametros
discount_factor=1.0
alpha=0.5
epsilon=0.1

#Inicializamos
Q = defaultdict(lambda: np.zeros(env.action_space.n))
# The policy we're following
policy = make_epsilon_greedy_policy(Q, epsilon, env.action_space.n)
# Reset the environment and pick the first action
state = env.reset()
t = 0



fatal: destination path 'reinforcement-learning' already exists and is not an empty directory.


In [17]:
# 4
env = OneStockEnv(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)
#env = GridworldEnv()
# env = OneStockEnv_vender_porcion_porc_comis(data = data.values,time_horizon = 20,porcentaje_comision=0.0007)


Q, stats_episode_lengths, stats_episode_rewards = q_learning(env, 200)
import matplotlib.pyplot as plt
plt.figure()
plt.plot(stats_episode_lengths)
plt.figure()
plt.plot(stats_episode_rewards)

state [[[50.   ]
  [50.505]
  [55.375]
  [55.62 ]
  [52.48 ]
  [52.475]
  [54.05 ]
  [52.64 ]
  [51.15 ]
  [51.35 ]
  [49.595]
  [50.475]
  [50.505]
  [50.37 ]
  [50.37 ]
  [50.8  ]
  [53.315]
  [53.725]
  [55.28 ]
  [56.17 ]]]
observation[[[50.   ]
  [50.505]
  [55.375]
  [55.62 ]
  [52.48 ]
  [52.475]
  [54.05 ]
  [52.64 ]
  [51.15 ]
  [51.35 ]
  [49.595]
  [50.475]
  [50.505]
  [50.37 ]
  [50.37 ]
  [50.8  ]
  [53.315]
  [53.725]
  [55.28 ]
  [56.17 ]]]


TypeError: ignored

In [18]:
# Action State
Q

defaultdict(<function __main__.<lambda>()>, {})

In [ ]:
# 5
# Vamos a ver que ha aprendido!
done = False
G, reward = 0,0
state = env.reset()
while done != True:
        action = np.argmax(Q[state]) #1
        state2, reward, done, info = env.step(action) #2
        #Q[state,action] += alpha * (reward + np.max(Q[state2]) - Q[state,action]) #3
        G += reward
        env._render()
        state = state2   
print(G)

T  o  o  o
o  o  o  o
o  o  o  o
o  o  x  T
T  o  o  o
o  o  o  o
o  o  o  o
o  o  o  x
-2.0


In [ ]:
p = np.zeros(16,)
for s in np.arange(0,16):
    p[s] = np.argmax(Q[s][:])

In [ ]:
print(np.resize(p,(4,4)))

[[0. 3. 3. 2.]
 [0. 3. 2. 2.]
 [0. 2. 2. 2.]
 [0. 1. 1. 0.]]
